In [1]:
# import sys
# import os
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dos_cart_product
from quantfreedom.strategies.strategy import (
    strat_bt_create_ind,
    strat_evaluate,
    strat_get_current_ind_settings,
    strat_get_ind_set_str,
    strat_get_total_ind_settings,
)
from quantfreedom.simulate import sim_df_backtest
from quantfreedom.sim_order_records import sim_get_or
import requests

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2

In [3]:
candles_np = mufex_main.get_candles_df(symbol="BTCUSDT", timeframe="5m", candles_to_dl=1000)

It took 00 mins and 01 seconds to download 1000 candles


In [4]:
mufex_main.set_exchange_settings(
    symbol="BTCUSDT",
    position_mode=PositionModeType.HedgeMode,
    leverage_mode=LeverageModeType.Isolated,
)
backtest_settings = BacktestSettings(
    divide_records_array_size_by=15,
    gains_pct_filter=-90,
    total_trade_filter=60,
    upside_filter=-np.inf,
)
dos_arrays = DynamicOrderSettingsArrays(
    entry_size_asset=np.array([0]),
    max_equity_risk_pct=np.array([6, 9, 12]),
    max_trades=np.array([5]),
    num_candles=np.array([100]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 4, 6]),
    sl_based_on_add_pct=np.array([0.01, 0.06]),
    sl_based_on_lookback=np.array([10, 30]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    sl_to_be_ze_type=np.array([0]),
    static_leverage=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2, 4]),
)
static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    long_or_short=LongOrShortType.Long,
    logger_bool=True,
    pg_min_max_sl_bcb=PriceGetterType.Min,
    sl_to_be_bool=False,
    z_or_e_type=ZeroOrEntryType.Nothing,
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    tp_fee_type=TakeProfitFeeType.Limit,
    trail_sl_bool=True,
)
dos_cart_arrays = dos_cart_product(
    dos_arrays=dos_arrays,
)

ExchangeSettings(
    limit_fee_pct = 0.0003,
    max_leverage = 150.0,
    market_fee_pct = 0.0009,
    mmr_pct = 0.004,
    min_leverage = 1.0,
    max_asset_size = 100.0,
    min_asset_size = 0.001,
    asset_tick_step = 3,
    position_mode = 1,
    leverage_mode = 1,
    price_tick_step = 1,
    leverage_tick_step = 2,
)


In [5]:
sim_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 216
Total combinations of settings to test: 1,296
Total candles: 1,000
Total candles to test: 1,296,000


,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,to_the_upside,fees_paid,total_pnl,ending_eq
0,0,0,75.0,17,58,-69.72,22.67,0.0,961.336,-697.2025,302.7975
1,0,3,65.0,16,49,-62.24,24.62,0.0,842.301,-622.4430,377.5570
2,0,24,65.0,13,52,-43.78,20.00,0.0,1199.843,-437.7622,562.2378
3,0,48,77.0,14,63,-61.58,18.18,0.0,1251.530,-615.7824,384.2176
4,0,60,67.0,14,53,-49.44,20.90,0.0,1052.824,-494.3776,505.6224
...,...,...,...,...,...,...,...,...,...,...,...
56,3,96,66.0,13,53,-56.96,19.70,0.0,1461.542,-569.6014,430.3986
57,3,120,66.0,13,53,-53.15,19.70,0.0,1509.963,-531.4890,468.5110
58,3,144,68.0,15,53,-76.86,22.06,0.0,1222.142,-768.6393,231.3607
59,3,168,63.0,12,51,-73.46,19.05,0.0,1312.187,-734.6142,265.3858


In [6]:
order_records_df = sim_get_or(
    backtest_settings=backtest_settings,
    candles=candles_np,
    dos_cart_arrays=dos_cart_arrays,
    evaluate=strat_evaluate,
    exchange_settings=mufex_main.exchange_settings,
    get_current_ind_settings=strat_get_current_ind_settings,
    get_ind_set_str=strat_get_ind_set_str,
    get_total_ind_settings=strat_get_total_ind_settings,
    ind_creator=strat_bt_create_ind,
    logger_type=LoggerType.Pass,
    starting_equity=1000.0,
    static_os=static_os,
    ind_set_index=3,
    dos_index=0,
)

strategy.py - get_current_ind_settings() - Created indicator settings
simulate.py - run_backtest() - Indicator settings index=3
strategy.py - strat_get_ind_set_str() - RSI Period= 20 RSI is below= 50.000
simulate.py - run_backtest() - Dynamic Order settings index=0
candle body str
candle body str
candle body str
candle body str
simulate.py - run_backtest() - Created Dynamic Order Settings part 1
entry_size_asset= 0.0
max_equity_risk_pct= 6.000
max_trades= 5
num_candles= 100
risk_account_pct_size= 3.000
risk_reward= 2.000
sl_based_on_add_pct= 0.010
sl_based_on_lookback= 10
sl_bcb_type= Low
sl_to_be_cb_type= Timestamp
sl_to_be_when_pct= 0.0
sl_to_be_ze_type= ZeroLoss
static_leverage= 0.0
trail_sl_bcb_type= Low
trail_sl_by_pct= 0.500
trail_sl_when_pct= 1.000
simulate.py - run_backtest() - Starting Bar=99
simulate.py - run_backtest() - account state order results pnl array all set to default



simulate.py - run_backtest() - ind_idx=3 dos_idx=0 bar_idx=99 timestamp= 1698063600000
simulate.

In [7]:
data = {
    "candles": candles_np.tolist(),
}
timestamp_list = candles_np[:, 0].tolist()

temp_entries_df = order_records_df[
    ["order_status", "timestamp", "average_entry", "entry_price", "liq_price", "sl_price", "tp_price"]
]
entries_df = temp_entries_df[temp_entries_df["order_status"] == "EntryFilled"]
entries_list_df = entries_df.values[:, 1:].tolist()

entries_list = np.vstack(candles_np[:, 0]).tolist()
sl_list = np.vstack(candles_np[:, 0]).tolist()
tp_list = np.vstack(candles_np[:, 0]).tolist()

for idx, timestamp in enumerate(timestamp_list):
    if timestamp == entries_list_df[0][0]:
        current_entry = entries_list_df[0]
        entries_list[idx] = [current_entry[0], current_entry[2]]
        sl_list[idx] = [current_entry[0], current_entry[4]]
        tp_list[idx] = [current_entry[0], current_entry[5]]
        del entries_list_df[0]
        if len(entries_list_df) == 0:
            break
data['entries'] = entries_list
data["sl_prices"]= sl_list
data["tp_prices"]= tp_list

In [8]:
temp_sl_filled_df = order_records_df[["order_status", "timestamp", "exit_price"]]
sl_filled_df = temp_sl_filled_df[temp_sl_filled_df["order_status"] == "StopLossFilled"]
sl_filled_list_df = sl_filled_df.values[:, 1:].tolist()
filled_sl_list = np.vstack(candles_np[:, 0]).tolist()

for idx, timestamp in enumerate(timestamp_list):
    if timestamp == sl_filled_list_df[0][0]:
        filled_sl_list[idx] = sl_filled_list_df[0]
        del sl_filled_list_df[0]
        if len(sl_filled_list_df) == 0:
            break
data["sl_filled"] = filled_sl_list

In [9]:
temp_tp_filled_df = order_records_df[["order_status", "timestamp", "exit_price"]]
tp_filled_df = temp_tp_filled_df[temp_tp_filled_df["order_status"] == "TakeProfitFilled"]
tp_filled_list_df = tp_filled_df.values[:, 1:].tolist()
filled_tp_list = np.vstack(candles_np[:, 0]).tolist()

for idx, timestamp in enumerate(timestamp_list):
    if timestamp == tp_filled_list_df[0][0]:
        filled_tp_list[idx] = tp_filled_list_df[0]
        del tp_filled_list_df[0]
        if len(tp_filled_list_df) == 0:
            break
data["tp_filled"] = filled_tp_list

In [11]:
res = requests.post("http://127.0.0.1:7779/plot", json=data)

# Convert response data to json
returned_data = res.json()

print(returned_data)

{'ok': True}
